In [3]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import tensorflow as tf
import pandas as pd

In [5]:
print(tf.__version__)
print(pd.__version__)
print(np.__version__)

1.1.0
0.20.3
1.13.1


## Load data

In [6]:
DATASET_PATH = "./dataset"

In [7]:
activities = {
    "WALKING":0,
    "WALKING_UPSTAIRS":1,
    "WALKING_DOWNSTAIRS":2,
    "SITTING":3,
    "STANDING":4,
    "LAYING":5
}

In [8]:
train = pd.read_csv(DATASET_PATH + "/train.csv")
test = pd.read_csv(DATASET_PATH + "/test.csv")

In [9]:
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [10]:
train['Activity'] = train['Activity'].astype("category", categories=activities)
test['Activity'] = test['Activity'].astype("category", categories=activities)

In [11]:
har_X_train = train.iloc[:,:561].values
har_y_train = train['Activity'].cat.codes.astype("int32").as_matrix()

X_test = test.iloc[:,:561].values
y_test = test['Activity'].cat.codes.astype("int32").as_matrix()

In [12]:
har_X_train[:5]

array([[ 0.28858451, -0.02029417, -0.13290514, ..., -0.84124676,
         0.17994061, -0.05862692],
       [ 0.27841883, -0.01641057, -0.12352019, ..., -0.8447876 ,
         0.18028889, -0.05431672],
       [ 0.27965306, -0.01946716, -0.11346169, ..., -0.84893347,
         0.18063731, -0.04911782],
       [ 0.27917394, -0.02620065, -0.12328257, ..., -0.84864938,
         0.18193476, -0.04766318],
       [ 0.27662877, -0.01656965, -0.11536185, ..., -0.84786525,
         0.18515116, -0.04389225]])

In [13]:
har_y_train[:5]

array([4, 4, 4, 4, 4], dtype=int32)

In [14]:
har_X_train.shape, har_y_train.shape

((7352, 561), (7352,))

In [15]:
X_test.shape, y_test.shape

((2947, 561), (2947,))

## Prepare data

In [16]:
#har_X_train_normed = normalize(har_X_train)
#har_X_test_normed = normalize(har_X_test)

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train,test_size=0.3, random_state=42)

In [18]:
X_train.shape, y_train.shape

((5146, 561), (5146,))

In [19]:
X_valid.shape

(2206, 561)

# Simple Dense model

## Hyperparameters

In [20]:
feature_len = 561
batch_size = 200
learning_rate = 0.001
n_epochs = 40

# outputs
n_classes = 6

In [21]:
n_batches = X_train.shape[0] // batch_size

## the model

In [22]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, feature_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")


In [23]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 300, activation=tf.nn.relu, name="dense1")
    dense2 = tf.layers.dense(dense1, 100, activation=tf.nn.relu, name="dense2")

    logits = tf.layers.dense(dense2, n_classes)    

In [24]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [25]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_dnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.892112
Epoch:  10 Valid accuracy:  0.966909
Epoch:  20 Valid accuracy:  0.974615
Epoch:  30 Valid accuracy:  0.974161


In [26]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_dnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_dnn_model.ckpt
Valid accuracy:  0.961469
Test Accuracy:  0.922972


## Dense model with L2 norm regularization, dropout and batch norm

In tensorflow we use `tf.layers.batch_normalization` for batch normalization
  - first apply the function to each dense layer, including logits layer
  - second need training the `tf.GraphKeys.UPDATE_OPS` after optimizer

fine tuning the model:
  1. training much slowly after batch norm added
  2. use `elu` to replace `relu`, but slow and worse accuracy
  3. use `batch_size` 600, but got worse accuracy

In [27]:
dropout_rate = 0.5

In [28]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, feature_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [29]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 256, name="dense1", kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn1 = tf.layers.batch_normalization(dense1, training=training)
    bn1_act = tf.nn.elu(bn1)
    drop1 = tf.layers.dropout(bn1_act, dropout_rate, training=training)
    
    dense2 = tf.layers.dense(drop1, 512, name="dense2", kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn2 = tf.layers.batch_normalization(dense2, training=training)
    bn2_act = tf.nn.elu(bn2)
    drop2 = tf.layers.dropout(bn2_act, dropout_rate, training=training)
    
    dense3 = tf.layers.dense(drop2, 256, name="dense3", kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn3 = tf.layers.batch_normalization(dense3, training=training)
    bn3_act = tf.nn.elu(bn3)
    drop3 = tf.layers.dropout(bn3_act, dropout_rate, training=training)
    
    logits_before_bn = tf.layers.dense(drop2, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)


In [30]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")
    #loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):   
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9).minimize(loss) 

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [31]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training:True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_dnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.422484
Epoch:  10 Valid accuracy:  0.894379
Epoch:  20 Valid accuracy:  0.925657
Epoch:  30 Valid accuracy:  0.940617


In [32]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_dnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_dnn_model.ckpt
Valid accuracy:  0.949683
Test Accuracy:  0.937564


## Dense model with early stopping

In [33]:
n_epochs = 800
dropout_rate = 0.5

In [34]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, feature_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [35]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 256, name="dense1", kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn1 = tf.layers.batch_normalization(dense1, training=training)
    bn1_act = tf.nn.elu(bn1)
    drop1 = tf.layers.dropout(bn1_act, dropout_rate, training=training)
    
    dense2 = tf.layers.dense(drop1, 512, name="dense2", kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn2 = tf.layers.batch_normalization(dense2, training=training)
    bn2_act = tf.nn.elu(bn2)
    drop2 = tf.layers.dropout(bn2_act, dropout_rate, training=training)
    
    dense3 = tf.layers.dense(drop2, 256, name="dense3", kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn3 = tf.layers.batch_normalization(dense3, training=training)
    bn3_act = tf.nn.elu(bn3)
    drop3 = tf.layers.dropout(bn3_act, dropout_rate, training=training)
    
    logits_before_bn = tf.layers.dense(drop2, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)


In [36]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")
    #loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):   
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9).minimize(loss) 

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [37]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_process=50

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc, loss_val =  sess.run([training_op, loss], feed_dict={X: X_batch, y: y_batch, training:True})
        
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=3) < best_loss:
                saver.save(sess, "./model/har_dnn_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_process:
                    break
                    
    print("Best Loss: ", best_loss)

Epoch:  0 Loss:  1.63477 Valid accuracy:  0.192656
Epoch:  5 Loss:  0.933518 Valid accuracy:  0.823663
Epoch:  10 Loss:  0.660262 Valid accuracy:  0.886673
Epoch:  15 Loss:  0.50991 Valid accuracy:  0.907072
Epoch:  20 Loss:  0.426819 Valid accuracy:  0.920218
Epoch:  25 Loss:  0.381947 Valid accuracy:  0.927924
Epoch:  30 Loss:  0.347488 Valid accuracy:  0.93563
Epoch:  35 Loss:  0.320802 Valid accuracy:  0.93971
Epoch:  40 Loss:  0.299311 Valid accuracy:  0.94515
Epoch:  45 Loss:  0.279635 Valid accuracy:  0.94787
Epoch:  50 Loss:  0.260266 Valid accuracy:  0.954669
Epoch:  55 Loss:  0.24651 Valid accuracy:  0.958296
Epoch:  60 Loss:  0.23803 Valid accuracy:  0.959656
Epoch:  65 Loss:  0.22164 Valid accuracy:  0.964642
Epoch:  70 Loss:  0.214306 Valid accuracy:  0.965095
Epoch:  75 Loss:  0.208207 Valid accuracy:  0.966002
Epoch:  80 Loss:  0.197945 Valid accuracy:  0.967815
Epoch:  85 Loss:  0.192705 Valid accuracy:  0.969628
Epoch:  90 Loss:  0.185155 Valid accuracy:  0.971442
Epoc

In [38]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_dnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_dnn_model.ckpt
Valid accuracy:  0.982321
Test Accuracy:  0.960638
